<a href="https://colab.research.google.com/github/mozz99/wirelessinsite/blob/main/codigos/dados_modelos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
def float_intercept(dados):
  d = dados['Distance(m)']
  pl = dados['Path_Loss(dB)']

# pl = prx

  # Cálculo Beta
  logd = 10 * np.log10(d)
  b = (np.sum(logd)) ** 2
  a = np.sum(logd ** 2)
  denbeta = a - (b / len(logd))
  numbeta = np.sum((logd * pl)) - ((np.sum(logd) * np.sum(pl)) / len(logd))
  beta = numbeta / denbeta

  # Cálculo alfa
  y = np.sum(pl) / len(pl)
  x = np.sum(logd) / len(logd)
  alfa = y - beta * x

  # Float Intercept
  fi = alfa + 10 * beta * np.log10(d)

  # Cálculo do Erro Quadrático Médio (MSE)
  MSE = np.sqrt(np.sum((pl - fi) ** 2) / len(pl))
  print("Rmse: ",MSE)
  return fi

In [ ]:
def close_in(dados):
  d0 = 1  # Distância de referência em metros

  y = (3 * 10 ** 8) / (3.5 * 10 ** 9)

  # Separando as colunas
  d = dados['Distance(m)']
  pt = 51
  L = dados['Path_Loss(dB)']


  # Definição das variáveis de entrada
  d0 = 1
  prxo = pt - L[0]

  Lo = 20 * np.log10((4 * np.pi) / y)  # path loss da primeira medida
  D = 10 * np.log10(d / d0)
  D = np.array(D)
  n = np.linalg.lstsq(D.reshape(-1, 1), (L - Lo), rcond=None)[0]  # regressão linear

  # Cálculo da potência do sinal recebido
  ci = 20 * np.log10((4 * np.pi * d0) / y) + 10 * n[0] * np.log10(d / d0)
  MSE = np.sqrt(np.sum((L - ci) ** 2) / len(L))
  print("Rmse: ",MSE)
  print("PLE: ", n[0])
  return ci

In [ ]:
def plot_model(model_1,model_2,dados,title):
  d = dados['Distance(m)']
  pl = dados['Path_Loss(dB)']

  plt.figure(figsize=(12,6))
  plt.plot(d, model_1,'-b', label=f'modelo fi')
  plt.plot(d,model_2,'-r',label='modelo ci')
  plt.scatter(d, pl,label='dados simulados')  # Use 'o-' para plotar pontos conectados por linhas
  plt.grid(True)
  plt.xticks(np.arange(17, 40, step=1))
  plt.xlabel('Distância (m)')
  plt.ylabel('Path Loss (dB)')
  plt.title('Modelos de Propagação ci x fi rota x')
  plt.legend()
  plt.title(f"{title}")
  plt.show()

In [ ]:
def diferença_sim_real(medidor1,simulador1,medidor2,simulador2):
  a = np.arange(18.0991,38.8226,0.58)
  b = np.arange(19.0286, 39.2583,0.75)
  simulador1['Distance(m)'],medidor1['Distance(m)'] = a,a
  simulador2['Distance(m)'],medidor2['Distance(m)'] = b,b
  MSE_1 = np.sqrt(np.sum((medidor1['Path_Loss(dB)'] - simulador1['Path_Loss(dB)']) ** 2) / len(a))
  MSE_2 = np.sqrt(np.sum((medidor2['Path_Loss(dB)'] - simulador2['Path_Loss(dB)']) ** 2) / len(b))
  return MSE_1,MSE_2

In [ ]:
a = np.arange(18.0991,38.8226,0.58)
b = np.arange(19.0286, 39.2583,0.75)


medidor1 = pd.read_csv('medido1.csv')
medidor2 = pd.read_csv('medido2.csv')

plt.scatter(medidor1['Distance(m)'],medidor1['Path_Loss(dB)'],label='rota 1')
plt.scatter(medidor2['Distance(m)'],medidor2['Path_Loss(dB)'],label='rota 2')
plt.legend()
plt.title('dados medidos')
plt.grid()
plt.show()

In [ ]:
sim_r1 = 'SETUPHALL.pl.t001_09.r016.p2m'
sim_r2 = 'SETUPHALL.pl.t001_09.r017.p2m'
# Carregar o arquivo de texto usando o separador padrão (espaço)

convert1 = pd.read_csv(sim_r1, sep='\s+')
convert2 = pd.read_csv(sim_r2, sep='\s+')

# Salvar DataFrame como CSV
convert1.to_csv('arquivo_convertido1.csv', index=False)
convert2.to_csv('arquivo_convertido2.csv', index=False)

simur1 = pd.read_csv('arquivo_convertido1.csv')
simur2 = pd.read_csv('arquivo_convertido2.csv')

simur1['Distance(m)'],medidor1['Distance(m)'] = a,a
simur2['Distance(m)'],medidor2['Distance(m)'] = b,b

In [ ]:
plt.scatter(simur1['Distance(m)'],simur1['Path_Loss(dB)'],label='rota 1')
plt.scatter(simur2['Distance(m)'],simur2['Path_Loss(dB)'],label='rota 2')
plt.title('dados simulados')
plt.legend()
plt.grid()
plt.show()

In [ ]:
fi1 = float_intercept(simur1)
ci1 = close_in(simur1)
plot_model(fi1,ci1,simur1,"rota 1")

fi2 = float_intercept(simur2)
ci2 = close_in(simur2)
plot_model(fi2,ci2,simur2,"rota 2")

plt.plot(simur1['Distance(m)'],simur1['Path_Loss(dB)'],'-ob',label='simulado')
plt.plot(medidor1['Distance(m)'],medidor1['Path_Loss(dB)'],'-*r',label='medido')
plt.legend()
plt.xlabel("Distancia (m)")
plt.ylabel("Path Loss (dB)")
plt.grid()
plt.title("sim x med rota 1")
plt.show()

In [ ]:
plt.plot(simur2['Distance(m)'],simur2['Path_Loss(dB)'],'-ob',label='simulado')
plt.plot(medidor2['Distance(m)'],medidor2['Path_Loss(dB)'],'-*r',label='medido')
plt.legend()
plt.title("sim x med rota 2")
plt.grid()
plt.xlabel("Distancia (m)")
plt.ylabel("Path Loss (dB)")
plt.show()
print(diferença_sim_real(simur1,medidor1,simur2,medidor2))